<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/10_schutz_personenbezogener_daten/schutz_personenbezogener_daten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schutz personenbezogener Kundendaten

Es lohnt sich, die Kundendaten der Versicherungsgesellschaft zu schützen. Es ist notwendig, eine solche Methode zur Datenkonvertierung zu entwickeln, damit es schwierig ist, persönliche Informationen wiederherzustellen und die Korrektheit ihrer Arbeit zu rechtfertigen.

Die Daten müssen so geschützt werden, dass die Qualität der maschinellen Lernmodelle bei der Konvertierung nicht beeinträchtigt wird.

# Datenexploration

In [1]:
# Wir importieren die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# Wir laden Tabellen mit Daten
df = pd.read_csv('/content/insurance.csv')

In [3]:
# Wir werden die erhaltenen Daten studieren
display(df.info())
display(df.shape)
display(df.head())
display(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

(5000, 5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


# Multiplikation von Matrizen

Wir werden die optimalen Gewichte für die Zeichen X und XP auswählen, indem wir die Trainingsformel transformieren

$w = (X^T X)^{-1} X^T$

$w'= ((XP)^{T}XP)^{-1}(XP)^{T} = (P^TX^TXP)^{-1} P^T X^T= P^{-1}(X^TX)^{-1}(P^T)^{-1}P^{T}X^T = P^{-1}(X^TX)^{-1}X^T = P^{-1} w$

Wir werden Vorhersagen treffen. Um dies zu tun, müssen wir die Merkmalsmatrix mit Gewichten multiplizieren.

$a = Xw$

$a' = XPw' = XPP^{-1}w = Xw = a$

Wir sehen, dass die Vorhersagen für die Zeichenmatrix X und für die Zeichenmatrix XP die gleichen sind

**Algorithmus**

1. Wir werden das Zielmerkmal aus der Quelltabelle hervorheben
2. Wir trainieren das Modell und berechnen die R2-Metrik
3. Wir wandeln die Merkmalstabelle in eine Matrix um
4. Wir werden eine zufällige reversible Matrix in der Größe 4x4 (nach der Anzahl der Merkmale) bilden
5. Wir erhalten eine neue Merkmalstabelle, indem wir die Merkmalsmatrix mit einer zufälligen Matrix multiplizieren
6. Wir trainieren das Modell und berechnen die R2-Metrik

**Begründung**

Zuvor haben wir theoretisch durch die Umwandlung der Lernformel bewiesen, dass sich die Vorabsagungen nach der Multiplikation der Merkmalsmatrix mit der reversiblen Matrix nicht ändern. Auf diese Weise wird sich die Qualität des Modells nicht ändern.

# Algorithmus überprüfen

In [4]:
# Wir werden das Zielmerkmal aus der Quelltabelle hervorheben
target = df['Страховые выплаты']
features = df.drop(['Страховые выплаты'] , axis=1)

In [10]:
# Wir trainieren das Modell und berechnen die R2-Metrik
model = LinearRegression()
model.fit(features, target)
print('R2-Metrik', model.score(features, target))

R2-Metrik 0.42494550286668


In [6]:
# Wir wandeln die Merkmalstabelle in eine Matrix um
features_matrix = features.values

In [7]:
# Wir bilden eine zufällige, reversible Matrix in der Größe 4x4 (nach Anzahl der Merkmale)
try:
    data_random = np.random.normal(size = (4,4)) 
except numpy.linalg.LinAlgError: 
    data_random = np.random.normal(size = (4,4)) 

In [8]:
# Wir erhalten eine neue Merkmalstabelle, indem wir die Merkmalsmatrix mit einer zufälligen Matrix multiplizieren
features_new = pd.DataFrame(features_matrix @ data_random)

In [11]:
# Wir trainieren das Modell und berechnen die R2-Metrik
model.fit(features_new, target)
print('R2-Metrik', model.score(features_new, target))

R2-Metrik 0.424945502866677


# Allgemeine Schlussfolgerung

Wir haben sichergestellt, dass sich die Qualität des Modells nach der Multiplikation der Merkmalsmatrix mit einer zufälligen Matrix nicht geändert hat, da die R2-Metrik in beiden Fällen gleich ist. Die persönlichen Daten der Kunden sind unbeschadet des Betriebs des Modells geschützt.